In [1]:
import torch
import sys
from torch import nn
import os
import random
import numpy as np
import pandas as pd

from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
import sklearn
from sklearn.metrics import roc_auc_score, log_loss, f1_score, confusion_matrix, classification_report
import timm
import cv2
from sklearn.preprocessing import LabelEncoder
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import warnings

In [3]:
dir_path = "../Data/carbon_reduction/else"

total_test_img_paths = []
for folder, subfolders, filenames in os.walk(dir_path):
    for img in filenames:
        total_test_img_paths.append(folder+'/'+img)
print(len(total_test_img_paths))

26074


In [4]:
## Pandas Test 데이터프레임 만들기
tst_df = pd.DataFrame(total_test_img_paths, columns=['image_id'])
tst_df['dir'] = tst_df['image_id'].apply(lambda x: os.path.dirname(x))
tst_df['image_id'] = tst_df['image_id'].apply(lambda x: os.path.basename(x))
test = tst_df
test

,image_id,dir
0,else25734.jpg,../Data/carbon_reduction/else
1,else22750.jpg,../Data/carbon_reduction/else
2,else5731.jpg,../Data/carbon_reduction/else
3,else5493.jpg,../Data/carbon_reduction/else
4,else29154.jpg,../Data/carbon_reduction/else
...,...,...
26069,else24476.jpg,../Data/carbon_reduction/else
26070,else11292.jpg,../Data/carbon_reduction/else
26071,else41223.jpg,../Data/carbon_reduction/else
26072,else3604.jpg,../Data/carbon_reduction/else


In [5]:
model_name = 'inception_resnet_v2'
model_dir = model_name + '/inception_resnet_v2_1'
%run inference.ipynb

100%|█████████████████████████████████████████| 204/204 [02:13<00:00,  1.53it/s]


In [8]:
encoded_labels = ["10kwalking","public_vehicle",'stair','pet','outlet','else', 'can', 'box', 'milk', 'cup']
le = LabelEncoder()
le.fit(encoded_labels)
test['pred'] = le.inverse_transform(test['pred'])
test


,image_id,dir,pred
0,else25734.jpg,../Data/carbon_reduction/else,else
1,else22750.jpg,../Data/carbon_reduction/else,else
2,else5731.jpg,../Data/carbon_reduction/else,else
3,else5493.jpg,../Data/carbon_reduction/else,else
4,else29154.jpg,../Data/carbon_reduction/else,else
...,...,...,...
26069,else24476.jpg,../Data/carbon_reduction/else,else
26070,else11292.jpg,../Data/carbon_reduction/else,else
26071,else41223.jpg,../Data/carbon_reduction/else,else
26072,else3604.jpg,../Data/carbon_reduction/else,else


### classifying ELSE images to 4 classes

In [38]:
tmp_list = []
temp = (test['pred'] == 'cup').value_counts()
temp

False    25743
True       331
Name: pred, dtype: int64